### Vector_DB 생성

In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers import ParentDocumentRetriever
from langchain_core.documents import Document
from langchain.storage import LocalFileStore, create_kv_docstore
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

api_key = os.environ.get('OPENAI_API_KEY')
embeddings = OpenAIEmbeddings(model='text-embedding-3-large', api_key=api_key)

vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory='../../vectors/parent_db',
    collection_name='parent_db',
)

base_path = os.path.abspath('../../vectors/parent_docstore')
os.makedirs(base_path, exist_ok=True)
parent_docstore = create_kv_docstore(LocalFileStore(base_path))

spliter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)
retriever = ParentDocumentRetriever(
    vectorstore=vector_store,
    docstore=parent_docstore,
    child_splitter=spliter,
    child_id_key='id',
    parent_id_key='parent_id'
)

### Vector_DB에 데이터 주입

In [ ]:
import os
import mysql.connector

mysql_pw = os.environ.get('MYSQL')
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password=mysql_pw,
    database='job_matcher'
)
cursor = conn.cursor(dictionary=True)

def fetch_job_postings(cursor):
    sql = """
    SELECT j.*
    FROM job_postings j
    LEFT JOIN embedded_ids e ON j.id = e.job_id
    WHERE e.job_id IS NULL
    LIMIT 100;
    """
    cursor.execute(sql)
    rows = cursor.fetchall()
    return rows

def add_job_posting_to_vectorstore(row, retriever, parent_store):
    parent_id = f"job_{row['id']}"
    parent_doc = Document(
        page_content=row['content'],
        metadata={
            "id": parent_id,
            "title": row['title'],
            "industry": row['industry'],
            "location": row['location'],
            "url": row['url'],
            "due_date": str(row['due_date'])
        }
    )
    parent_store.mset([(parent_id, parent_doc)])

    if row['content']:
        child = summary_content(row['content'])
    else:
        child = summary_content(generate_job_description(row))

    child_sections = {
        "근무조건": child['근무조건'],
        "복리후생": child['복리후생'],
        "자격조건": child['자격'],
        "전형절차": child['전형'],
        "기타": child['기타']
    }

    child_docs = []
    for idx, (category, content) in enumerate(child_sections.items()):
        if content:
            child_doc = Document(
                page_content=content,
                metadata={
                    "id": f"{parent_id}_{idx}",
                    "parent_id": parent_id,
                    "category": category,
                    "title": row['title'],
                    "career": row['career'],
                    "car_level": get_car_level(row['career']),
                    "education": row['education'],
                    "edu_level": get_edu_level(row['education']),
                    "skill": row['skill'],
                    "preferred": row['preferred'],
                    "location": row['location'],
                    "salary": row['salary'],
                    "workingtime": row['working_time'],
                    "url": row['url'],
                    'company_name': row['company_name'],
                    'founded': row['founded'],
                    "homepage": row['homepage'],
                }
            )
            child_docs.append(child_doc)

    if child_docs:
        retriever.add_documents(child_docs)
rows = fetch_job_postings(cursor)
for row in rows:
    add_job_posting_to_vectorstore(row, retriever, parent_docstore)
    insert_embedded_id(conn, row['id'])

conn.commit()
cursor.close()
conn.close()


### 학력 수치 , 경력 수치 추가

In [8]:
import re

def get_car_level(경력):
    if 경력 is None:
        return -1
    경력 = 경력.replace(" ", "").lower()

    if "무관" in 경력:
        return -1
    if "신입" in 경력 and "경력" not in 경력:
        return 0

    # 경력(X년이상) 패턴 매칭
    match = re.search(r"경력\(?(\d+)년", 경력)
    if match:
        return int(match.group(1))

    # 신입·경력(X년이상)
    match = re.search(r"신입·경력\(?(\d+)년", 경력)
    if match:
        return int(match.group(1))

    if "경력" in 경력:
        return 1  # 그냥 '경력'만 있을 때는 1년차 기본값
    if "신입·경력" in 경력:
        return 0  # 신입 가능인 경우

    return -1

def get_edu_level(학력):
    if 학력 is None:
        return -1
    학력 = 학력.replace(" ", "").lower()
    if "무관" in 학력:
        return -1
    elif "박사" in 학력:
        return 3
    elif "석사" in 학력 or "대학원" in 학력:
        return 2
    elif "대졸" in 학력 or "학사" in 학력:
        return 1
    elif "초대졸" in 학력 or "전문대" in 학력:
        return 0
    elif "고졸" in 학력:
        return -2  # 고졸 기준 따로 두고 싶으면 사용
    else:
        return -1

### 본문 내용 요약

In [3]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.schema.runnable import RunnableLambda
api_key = os.environ.get("OPENAI_API_KEY")

def summary_content(job_content):
    model = ChatOpenAI(
        model = 'gpt-4.1',
        temperature = 0.1
    )

    response_schemas = [
        ResponseSchema(name="근무조건", description="간결한 근무조건 요약"),
        ResponseSchema(name="복리후생", description="간결한 복리후생 요약"),
        ResponseSchema(name="자격", description="간결한 자격요건 요약"),
        ResponseSchema(name="전형", description="간결한 전형절차 요약"),
        ResponseSchema(name="기타", description="기타 지원방법과 문의처 요약")
    ]

    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()


    prompt = ChatPromptTemplate.from_template(
        """
        너는 채용공고 요약 전문가야.
        다음 채용공고 본문을 읽고, 주어진 형식에 따라 JSON으로 간결하게 요약해줘.
        {format_instructions}

        채용공고 본문:
        \"\"\"
        {job_content}
        \"\"\"
        """
    )

    chain = (
        prompt
        | model
        | RunnableLambda(lambda x: output_parser.parse(x.content))
    )

    result = chain.invoke({
        "format_instructions" : format_instructions,
        "job_content" : job_content
    })

    return result

### 본문이 없다면 본문 생성

In [4]:
def generate_job_description(data):
    lines = []
    lines.append(f"{data['company_name']}에서 {data['position_role']} 포지션을 모집합니다.")

    if data.get('contract_type') and data.get('location'):
        lines.append(f"본 포지션은 {data['contract_type']}으로, {data['location']}에서 근무하게 됩니다.")
    elif data.get('contract_type'):
        lines.append(f"본 포지션은 {data['contract_type']}입니다.")
    elif data.get('location'):
        lines.append(f"근무지는 {data['location']}입니다.")

    if data.get('career'):
        lines.append(f"경력 조건은 {data['career']}입니다.")
    if data.get('education'):
        lines.append(f"학력 조건은 {data['education']}입니다.")
    if data.get('working_time'):
        lines.append(f"근무 시간은 {data['working_time']}입니다.")
    if data.get('skill'):
        lines.append(f"요구되는 기술 스택은 {data['skill']}입니다.")
    if data.get('preferred'):
        lines.append(f"우대사항으로는 {data['preferred']}이 있습니다.")
    if data.get('salary'):
        lines.append(f"급여는 {data['salary']}입니다.")
    if data.get('due_date'):
        lines.append(f"접수 마감일은 {data['due_date']}입니다.")

    # 회사 소개
    company_info = []
    if data.get('founded'):
        company_info.append(f"{data['founded']}년에 설립")
    if data.get('employees'):
        company_info.append(f"현재 {data['employees']}의 직원")
    if data.get('company_type'):
        company_info.append(f"{data['company_type']}으로 분류")
    if data.get('industry'):
        company_info.append(f"산업 분야는 {data['industry']}")
    if company_info:
        lines.append(", ".join(company_info) + "입니다.")

    if data.get('homepage'):
        lines.append(f"자세한 정보는 회사 홈페이지({data['homepage']})에서 확인하실 수 있습니다.")

    return "\n".join(lines)


In [5]:
# def insert_embedded_id(cursor, job_id):
#     sql = """
#     INSERT INTO embedded_ids (job_id)
#     VALUES (%s)
#     ON DUPLICATE KEY UPDATE job_id = job_id;  -- 중복일 경우 무시
#     """
#     print("before insert embedded id")
#     cursor.execute(sql, (job_id,))
#     print('after insert embedded id')


In [5]:
def insert_embedded_id(conn, job_id):
    with conn.cursor() as cur:  # 커서 open
        sql = """
        INSERT INTO embedded_ids (job_id)
        VALUES (%s)
        ON DUPLICATE KEY UPDATE job_id = job_id;
        """
        print("before insert embedded id")
        cur.execute(sql, (job_id,))
        print('after insert embedded id')
